In [7]:
import warnings
import logging


warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [8]:
from utils import make_environments
from utils import pearl_utils
from configs  import defaults

from utils.reward_functions import log_reward_function,cumulative_reward_function,sharpe_reward_function
from utils. utils import make_hidden_dims
import optuna
from optuna.samplers import TPESampler
from neuralforecast.core import NeuralForecast
from Pearl.pearl.utils.instantiations.environments.gym_environment import GymEnvironment
from Pearl.pearl.utils.functional_utils.train_and_eval.online_learning import online_learning
import datetime

import numpy as np
import pickle
import boto3
import Keys

In [9]:
# s3=boto3.client('s3', aws_access_key_id=Keys.AWS_ACCESS_KEY, aws_secret_access_key=Keys.AWS_SECRET_KEY)
# agent_path=f'Agent/pearl_{defaults.model_name}_model.pkl'

# s3.download_file('coinbasetradehistory',f'pearl_{defaults.model_name}_model.pkl',agent_path)


In [10]:
from configs import defaults

In [11]:
# model=NeuralForecast.load('MultiHeadForecastingModel/')

In [12]:
reward_functions=[log_reward_function,cumulative_reward_function,sharpe_reward_function]
train_env,test_env=make_environments.make_envs(reward_function=log_reward_function)
study_name=f"{defaults.model_name}"
storage_name="sqlite:///PearlHPTuning.sqlite3"

test_env.observation_space.shape,train_env.action_space.n


Seed set to 12
Seed set to 5


['data_forex/oanda-USDJPY-1h.pkl']


100%|██████████| 78/78 [00:00<00:00, 11895.71it/s]
1it [00:00,  9.05it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

2024-11-04 12:00:00 2024-12-04 12:00:00


((24,), np.int64(2))

In [13]:
today=datetime.datetime.now().strftime('%Y-%m-%d')
study = optuna.load_study(study_name=study_name,
                            storage=storage_name,

                            )


In [14]:
# print(f"Best value: {study.best_value} (params: {study.best_params})")
best_trials=study.best_trials
best_trials

[FrozenTrial(number=45, state=TrialState.COMPLETE, values=[1191.6558529692093, 20.4], datetime_start=datetime.datetime(2024, 11, 24, 2, 9, 29, 249544), datetime_complete=datetime.datetime(2024, 11, 24, 2, 14, 53, 613837), params={'reward_function': 0, 'algorithm': 'dqn', 'n_layers': 1, 'n_units': 256, 'training_rounds': 21, 'learning_rate': 2.8764743562927777e-05, 'discount_factor': 0.911339518183664, 'batch_size': 64, 'target_update_freq': 5, 'soft_update_tau': 0.30917867193935245, 'is_conservative': False, 'lstm': True, 'conservative_alpha': 0.5255327383263214, 'learn_after_episode': True, 'learning_steps': 51, 'n_epochs': 100}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'reward_function': CategoricalDistribution(choices=(0, 1, 2)), 'algorithm': CategoricalDistribution(choices=('dqn', 'ddqn')), 'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units': CategoricalDistribution(choices=(64, 128, 256, 512)), 'training_rounds': IntDistribution(h

In [15]:


agent_path=f'Agent/pearl_{defaults.model_name}_model.pkl'

agent_path


'Agent/pearl_USDJPY_model.pkl'

In [16]:
best_agents={}
for i in range(len(best_trials)):
    agent,learning_params=pearl_utils.load_agent_from_study(study_path=storage_name,
                                            study_name=study_name,
                                            action_space_dim=train_env.action_space.n,
                                            observation_space_dim=train_env.observation_space.shape[0],
                                            version=i)
    
    agent,profit,n_trades=pearl_utils.train_production_agent(agent,
                                learning_params,
                                train_env=train_env,
                                test_env=test_env,
                                save_path=agent_path)
    best_agents[profit]=agent
agent=best_agents[max(best_agents)]
    

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [02:53<?, ?it/s]
Profit: 798.1510444987586, Number of Trades: 19.2: 100%|██████████| 100/100 [02:01<00:00,  1.22s/it]             


Testing Return AVG Profit: 798.1510444987586, AVG Number of Trades: 19.2


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [02:39<?, ?it/s]
Profit: 839.8773936343555, Number of Trades: 17.2: 100%|██████████| 100/100 [02:06<00:00,  1.27s/it]             


Testing Return AVG Profit: 839.8773936343555, AVG Number of Trades: 17.2


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [02:39<?, ?it/s]


In [17]:
# agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
#                                             study_name='pearl-2024-11-12-hp-search',
#                                             action_space_dim=2,
#                                             observation_space_dim=30,
#                                             version=1)

# agent=pearl_utils.train_production_agent(agent,
#                              learning_params,
#                              train_env=train_env,
#                              test_env=train_env,
#                              save_path=agent_path)

In [18]:
agent=best_agents[max(best_agents)]


In [19]:
best_agents,max(best_agents)


({np.float64(798.1510444987586): <Pearl.pearl.pearl_agent.PearlAgent at 0x32b03bb50>,
  np.float64(1205.5609220027036): <Pearl.pearl.pearl_agent.PearlAgent at 0x3bcfa3d60>,
  np.float64(1147.5101040437096): <Pearl.pearl.pearl_agent.PearlAgent at 0x3a498fd90>,
  np.float64(1118.9936164597516): <Pearl.pearl.pearl_agent.PearlAgent at 0x42896ff70>,
  np.float64(851.6922782859551): <Pearl.pearl.pearl_agent.PearlAgent at 0x3a49669b0>,
  np.float64(839.8773936343555): <Pearl.pearl.pearl_agent.PearlAgent at 0x49619b940>},
 np.float64(1205.5609220027036))

In [20]:
profit,n_trades=pearl_utils.test_pearl_model(agent,test_env)
print(f"Testing Return AVG Profit: {profit}, AVG Number of Trades: {n_trades}")

Profit: 1184.2370116392528, Number of Trades: 20.4: 100%|██████████| 100/100 [02:07<00:00,  1.27s/it]             

Testing Return AVG Profit: 1184.2370116392528, AVG Number of Trades: 20.4


In [22]:
from utils import save_utils


In [23]:
save_utils.save_agent(agent,agent_path)

Device: mps:0


In [ ]:
# pickle.dump(agent.policy_learner.state_dict(),open(agent_path,'wb'))

In [ ]:
# weights=pickle.load(open(agent_path,'rb'))

In [ ]:
# agent.policy_learner.load_state_dict(weights)

In [ ]:

# agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
#                                         study_name='pearl-2024-11-12-hp-search',
#                                         action_space_dim=2,
#                                         observation_space_dim=30)
# agent=load_agent_weights(agent,weight_path=agent_path)

In [ ]:
profit,n_trades=pearl_utils.test_pearl_model(agent,test_env)
print(f"Testing Return AVG Profit: {profit}, AVG Number of Trades: {n_trades}")